In [212]:
"""
Reading
"""
with open ("24817120.txt", "r") as myfile:
    data=myfile.readlines()
import re
import json

In [213]:
"""
6. pre pro same to original
7. requirement null

"""

"""
prerequistic --- single string, multiple list x
prohibition --- single string, multiple list x
chief --- single string, multiple list x
requirement --- single string, multiple list ?
outcome --- ?
"""

"""
Suggestion:
1. synopsis paragraph
2. "provision"
3. single & multiple in prerequistic, prohibition, chief
"""

'\nSuggestion:\n1. synopsis paragraph\n2. "provision"\n3. single & multiple in prerequistic, prohibition, chief\n'

In [214]:
"""
Parsing to unit
"""
i = 0
index = []
while i < len(data):
    hkg = re.findall('<span class="unitcode">\w*\d*</span>', data[i])
    if len(hkg) != 0:
        index.append(i)
    i+=1
g = 0
parsing = []
while g < len(index):
    if g < len(index)-1:
        parsing.append(data[index[g]:index[g+1]])
    else:
        parsing.append(data[index[g]:])
    g += 1


In [215]:
"""
Paring to paragraph
"""
i = 0
index_list = {}
while i < len(parsing):
    section_index = {}
    for index, x in enumerate(parsing[i]):
        if len(re.findall('<h1.*</h1>',x)) != 0:
            section_index["@id"]=index
            section_index["title"]=index
        elif len(re.findall('Chief examiner',x)) != 0:
            section_index["chief_examiners"]=index
        elif len(re.findall("Prerequisites",x)) != 0:
            section_index["prerequisites"]=index
        elif len(re.findall("Prohibisions",x)) != 0:
            section_index["prohibisions"] = index
        elif len(re.findall("Synopsis</h2>",x)) != 0:
            section_index["synopsis"] = index
        elif len(re.findall("Outcomes</h2>",x)) != 0:
            section_index["outcomes"] = index
        elif len(re.findall("Assessment</h2?",x)) != 0:
            section_index["requirement"] = index
        else:
            continue
    index_list["parsing"+str(i)]=section_index
    i += 1
h = 0
content_list = {}
while h < len(parsing):
    content = {}
    keys = list(index_list['parsing' + str(h)].keys())
    values = list(index_list['parsing' + str(h)].values())
    content[keys[0]]=parsing[h][values[0]]
    content[keys[1]]=parsing[h][values[1]]
    g = 2
    while g < len(keys):
        if g <len(keys)-1:
            content[keys[g]]=parsing[h][values[g]:values[g+1]]
        else:
            content[keys[g]]=parsing[h][values[g]:]
        g += 1
    content_list["parsing"+str(h)]=content
    h += 1

In [216]:
"""
id
"""
def id_identify(id_raw):
    id = re.findall('<span class="unitcode">\w*\d*</span>',id_raw)[0]
    id = html_tag_delete(id)   
    return id

In [217]:
"""
title
"""
def title_identify(title_raw):
    title = ''
    title = re.findall("\s-\s.*\s-\s",title_raw)[0]
    title = html_tag_delete(title)
    title = title.replace(' - ','')
    return(title)

In [277]:
"""
chief examiner
"""
def chief_examiner_identify(chief_examiner_raw):
    ce = chief_examiner_raw
    chief_examiners_parsing = []
    chief_examiners =[]
    stop_tag = ('Coordinator','Unit guides','Offered','Notes')
    stop_index = 0
    for index, x in enumerate(ce):
        for y in stop_tag:
            if len(re.findall(y,x)) != 0:
                stop_index = index
                break
            break
    for y in ce[0:stop_index]:
        if re.findall("staffsearch.*",y):
            y = y.replace('Chief examiner(s)','')
            chief_examiners_parsing.append(y)
    for z in chief_examiners_parsing:
        chief_examiners.append(html_tag_delete(z))
    
    return null_entry_handling(chief_examiners)

In [220]:
"""
synopsis
"""
def synopsis_identify(synopsis_raw):
    sy = synopsis_raw
    synopsis = ''
    for x in sy:
        if len(re.findall("<p>.*</p>",x)) !=0:
            synopsis = x
            break
    synopsis = html_tag_delete(synopsis)
    return(synopsis)

In [221]:
"""
outcome
"""
def outcome_identify(outcome_raw):
    oc = outcome_raw
    outcome_parsing = []
    outcome = []
    for x in oc:
        if len(re.findall('^<p>.*|^<li>.*',x)) != 0:
            outcome_parsing.append(re.findall('^<p>.*|^<li>.*',x)[0])
    for y in outcome_parsing:
        
        outcome.append(html_tag_delete(y))
    
    return null_entry_handling(outcome)

In [322]:
"""
requirement
"""
def requirement_identify(requirement_raw):
    req = requirement_raw
    requirement_parsing =[]
    requirements = []
    for x in req:
        if re.findall("</div>",x):
            requirement_parsing.append(x)
            break
        else:
            requirement_parsing.append(x)
    
    
    for y in requirement_parsing:
        y = y.replace('Assessment</h2>','')
        if len(y) != 0:
            requirements.append(html_tag_delete(y))
    

    
    
    return null_entry_handling(requirements)

In [225]:
"""
prerequisite
"""
def prerequisite_identify(prerequisite_raw):
    pre = prerequisite_raw
    prerequisite = ''
    for x in pre:
        prerequisite += x
    prerequisite = html_tag_delete(prerequisite)

    prerequisite = prerequisite.replace('Prerequisites','')

    prerequisite_code = []
    for y in re.findall('[A-Z]+\d+',prerequisite):
        if len(y) != 0:
            prerequisite_code.append(y)
    pre_code_set = set(prerequisite_code)
    prerequisite_code =list(pre_code_set)
    
    
    return null_entry_handling(prerequisite_code)

In [226]:
"""
prohibision
"""
def prohibision_identify(prohibision_raw):
    pro = prohibision_raw
    prohibision = ''
    for x in pro:
        prohibision += x
    prohibision = prohibision.replace('Prohibisions','')
    prohibision = html_tag_delete(prohibision)
    prohibision_code = []
    for y in re.findall('[A-Z]+\d+',prohibision):
        if len(y) != 0:
            prohibision_code.append(y)
    pro_code_set = set(prohibision_code)
    prohibision_code = list(pro_code_set)
    
    
    return null_entry_handling(prohibision_code)

In [227]:
"""
single multiple judge
"""

def single_multiple_judgement(single_multiple):
    if len (single_multiple) >1:
        return (single_multiple) 
    elif len (single_multiple) < 1:
        pass
    else:
        return (single_multiple[0])

In [321]:
"""
null entry handling
"""

def null_entry_handling(entry):
    result = []
    for x in entry:
        if len(x) != 0:
            result.append(x)
    return result


In [229]:
"""
html tag delete
"""
def html_tag_delete(material):
    

    html_start_tag = ['<span','<div','<p','<ul','<li','<h1','<h2','<br','<ul','<nav','<ol','<a']
    html_end_tag = ['</span>','</div>','</p>','</ul>','</li>','</h1>','</h2>','</br>','</ul>','</nav>','</ol>','</a>','\n']
    html_start_tag_re = '.*?>'
    html_start_reg = []
    for x in html_start_tag:
        reg = x+html_start_tag_re
        if len(re.findall(reg,material)) != 0:
            html_start_reg.append(re.findall(reg,material))
    for x in html_start_reg:
        for y in x:
            material = material.replace(y,'')
    for n in html_end_tag:
        material = material.replace(n,'')
    return material

In [230]:
file = open('xml_out.xml', 'w')

file.write("<?xml version='1.0' encoding='utf8'?>\n")
file.write("<units>\n")
file.close()

In [238]:
i = 0
unit_information = {}
units = {}
units["unit"] = []

file = open('xml_out.xml', 'a')

while i < len(content_list):
    
    parsing_number = 'parsing' + str(i)
    key = content_list[parsing_number].keys()
    par = content_list[parsing_number]

    entry_id = "NA"
    entry_title = "NA"
    entry_chief = "NA"
    entry_prerequisites = "NA"
    entry_prohibisions = "NA"
    entry_synopsis = "NA"
    entry_outcomes = "NA"
    entry_requirement = "NA"

    
    
    entry = {

        "@id": "NA",
        "title": "NA",
        "synopsis": "NA",
        "pre_requistics": "NA",
        "prohibisions":"NA",
        "requirements": "NA",
        "outcomes": "NA",
        "chief_examiners": "NA"
            
                
    }
    for x in key:
        if x == '@id':
            entry_id = id_identify(par['@id'])
            entry["@id"] = single_multiple_judgement(id_identify(par['@id']))
        elif x == 'title':
            entry_title = title_identify(par["title"])
            entry["title"] = single_multiple_judgement(title_identify(par["title"]))
        elif x == 'chief_examiners':
            entry_chief = chief_examiner_identify(par["chief_examiners"])
            chief = {}
            chief["chief_examiner"] = single_multiple_judgement(chief_examiner_identify(par["chief_examiners"]))
            entry["chief_examiners"] = chief
        elif x == 'prerequisites':
            entry_prerequisites = prerequisite_identify(par["prerequisites"])
            pre = {}
            pre["pre_requistic"] = single_multiple_judgement(prerequisite_identify(par["prerequisites"]))
            entry["pre_requistics"] = pre
        elif x == 'prohibisions':
            entry_prohibisions = prohibision_identify(par["prohibisions"])
            pro = {}
            pro ["prohibision"] = single_multiple_judgement(prohibision_identify(par["prohibisions"]))
            entry["prohibisions"] = pro
        elif x == 'synopsis':
            entry_synopsis = synopsis_identify(par['synopsis'])
            entry["synopsis"] = single_multiple_judgement(synopsis_identify(par['synopsis']))
        elif x == 'outcomes':
            entry_outcomes = outcome_identify(par['outcomes'])
            out = {}
            out ["outcome"] = single_multiple_judgement(outcome_identify(par['outcomes']))
            entry["outcomes"] = out
        else:
            entry_requirement = requirement_identify(par['requirement'])
            ent = {}
            ent ["requirement"] = single_multiple_judgement(requirement_identify(par['requirement']))
            entry["requirements"] = ent
    
    
    
    file.write("<unit id='"+ entry_id +"'>\n")
    
    file.write("<title>" + entry_title + "</title>\n")
    
    file.write("<synopsis>" + entry_synopsis + "</synopsis>\n")
    
    
    
    if entry_prerequisites == "NA":
        file.write("<pre_requistics> NA </pre_requistics>\n")
    else:
        file.write("<pre_requistics>\n")
        j = 0
        while j < len(entry_prerequisites):
            file.write("<pre_requistic> " + entry_prerequisites[j] + "</pre_requistic>")
            j += 1
        file.write("</pre_requistics>\n")
        
        
    if entry_prohibisions == "NA":
        file.write("<prohibisions> NA </prohibisions>\n")
    else:
        file.write("<prohibisions>\n")
        j = 0
        while j < len(entry_prohibisions):
            file.write("<prohibision> " + entry_prohibisions[j] + "</prohibision>")
            j += 1
        file.write("</prohibisions>\n")
    
    
    if entry_requirement == "NA":
        file.write("<requirements> NA </requirements>\n")
    else:
        file.write("<requirements>\n")
        j = 0
        while j < len(entry_requirement):
            file.write("<requirement> " + entry_requirement[j] + "</requirement>")
            j += 1
        file.write("</requirements>\n")

    
    if entry_outcomes == "NA":
        file.write("<outcomes> NA </outcomes>\n")
    else:
        file.write("<outcomes>\n")
        j = 0
        while j < len(entry_outcomes):
            file.write("<outcome> " + entry_outcomes[j] + "</outcome>")
            j += 1
        file.write("</outcomes>\n")
        
        
        
    if entry_chief == "NA":
        file.write("<chief_examiners> NA </chief_examiners>\n")
    else:
        file.write("<chief_examiners>\n")
        j = 0
        while j < len(entry_chief):
            file.write("<chief_examiner> " + entry_chief[j] + "</chief_examiner>")
            j += 1
        file.write("</chief_examiners>\n")
        
        
        
    file.write("</unit>\n")

    
    units["unit"].append(entry)
    i += 1
unit_information["units"]=units



file.write("</units>\n")
file.close()

In [239]:
with open('json_out.json', 'w') as outfile:
    json.dump(unit_information, outfile, indent = 4)